### Pydantic
pydantic models provide the richest feature set with field validation, descriptions and nested structures

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model =init_chat_model("groq:qwen/qwen3-32b")

In [2]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002023623C1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000202363A4590>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")



In [4]:
model_with_structured_output = model.with_structured_output(Movie)
model_with_structured_output

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002023623C1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000202363A4590>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'dire

In [5]:
model_with_structured_output.invoke("Provide details of Movie Bahubali2")

Movie(title='Bahubali 2', year=2017, director='S.S. Rajamouli', rating=8.5)

In [6]:
model_with_structured_output.invoke("Provide details of Movie RRR")

Movie(title='RRR', year=2022, director='SS Rajamouli', rating=8.0)

In [7]:
class Movie(BaseModel):
    """A Movie with details"""
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

model_with_structure = model.with_structured_output(Movie,include_raw=True)

response = model_with_structure.invoke("Provide Details about Movie Bahubali")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Bahubali. Let me check the tools provided. There\'s a Movie function that requires title, year, director, and rating. I need to fill these parameters. I remember that Bahubali is a popular Indian film. The title is "Bahubali: The Beginning" for the first part. The release year was 2015. The director is S.S. Rajamouli. The rating on IMDb is around 8.1. Let me confirm these details to make sure. Yes, that\'s correct. So I\'ll structure the tool call with these parameters.\n', 'tool_calls': [{'id': '7v66h6mtd', 'function': {'arguments': '{"director":"S.S. Rajamouli","rating":8.1,"title":"Bahubali: The Beginning","year":2015}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 188, 'prompt_tokens': 229, 'total_tokens': 417, 'completion_time': 0.347953214, 'completion_tokens_details': {'reasoning_tokens': 131}, 'prompt_tim

### Nested Structure

In [8]:
class Actor(BaseModel):
    name:str
    role:str


class MovieDetails(BaseModel):
    title:str
    year:str
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure  = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide Details of Movie Bahubali2")
response

MovieDetails(title='Bahubali 2: The Conclusion', year='2017', cast=[Actor(name='Prabhas', role='Bahubali'), Actor(name='Tamannaah Bhatia', role='Devasena'), Actor(name='Sathyaraj', role='Bhallala Deva')], genres=['Action', 'Adventure', 'Fantasy'], budget=100.0)

### Typed Dict

This provides a simpler alternative using python's built-in typing, ideal when you don't need run time validation

In [9]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A Movie with details)"""
    title:Annotated[str,...,"The title of the movie"]
    year:Annotated[int,...,"This year movie was released"]
    director:Annotated[str,...,"The director of the movie"]
    rating:Annotated[float,...,"The movies rating out of 10"]

model_with_typed_dict = model.with_structured_output(MovieDict)
response = model_with_typed_dict.invoke("Please provide the details of movie Bahubali")

response


{'director': 'S.S. Rajamouli',
 'rating': 8,
 'title': 'Bahubali: The Beginning',
 'year': 2015}

In [10]:
class Actor(TypedDict):
    name:str
    role:str


class MovieDetails(TypedDict):
    title:str
    year:str
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure  = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide Details of Movie Bahubali2")
response

{'budget': 2500000000,
 'cast': [{'name': 'Prabhas', 'role': 'Shivudu / Bijjaladeva II'},
  {'name': 'Rana Daggubati', 'role': 'Bhallaladeva'},
  {'name': 'Tamannaah Bhatia', 'role': 'Devasena'},
  {'name': 'Sathyaraj', 'role': 'Amarendra Baahubali'}],
 'genres': ['Action', 'Adventure', 'Drama'],
 'title': 'Bahubali 2: The Conclusion',
 'year': '2017'}

### Data classes

In [ ]:
from langchain.agents import create_agent
class ContactInfo(BaseModel):
    """Contact information for a person"""
    name:str = Field(description="The name of the person")
    email:str = Field(description="The email address of the person")
    phone:str = Field(description="The phone number of the person")

model =init_chat_model("groq:qwen/qwen3-32b")
agent = create_agent(
    model=model,
    response_format=ContactInfo
)

result = agent.invoke({
    "messages":[{"role":"user","content":"Extract info from: siddu, siddu@gmail.com, 9945612355"}]
})

print(result["structured_response"])

name='siddu' email='siddu@gmail.com' phone='9945612355'


In [12]:
result

{'messages': [HumanMessage(content='Extract info from: siddu, siddu@gmail.com, 9945612355', additional_kwargs={}, response_metadata={}, id='1f9dba1b-f960-40cc-bed3-2a43bd17d6e1'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let me see. The user provided a string with some information: "siddu, siddu@gmail.com, 9945612355". They want me to extract the info and use the ContactInfo function.\n\nFirst, I need to split the string into parts. The name is "siddu", the email is "siddu@gmail.com", and the phone number is "9945612355". Let me check if all required fields are present. The function requires name, email, and phone. Yes, all three are there. \n\nWait, the phone number is a number, but the input has it as a string. The function parameters expect a string type for phone, so that\'s okay. No need to convert. \n\nI should structure the JSON object with the parameters. The name is "siddu", email is "siddu@gmail.com", phone is "9945612355". Let me double-check th

In [13]:
### Data class

from dataclasses import dataclass

@dataclass
class ContactInfo:
    """Contact information of a person"""
    name:str
    email:str
    phone:str


model =init_chat_model("groq:qwen/qwen3-32b")
agent = create_agent(
    model=model,
    response_format=ContactInfo
)

result = agent.invoke({
    "messages":[{"role":"user","content":"Extract info from: siddu, siddu@gmail.com, 9945612355"}]
})

print(result["structured_response"])

ContactInfo(name='siddu', email='siddu@gmail.com', phone='9945612355')
